In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
from astropy import units
import os
import astropy.units as u
import matplotlib.pyplot as plt

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

# Radial distribution of satellites in the Local Group

Count the accumulation of satellites with radius from the centres of potential of the MW and M31 galaxies **together**.

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

## Get data for plotting

Choose how to distinguish between satellites and isolated:

In [ ]:
distinction = "by_GN"

Read the datasets from each simulation, convert the units and split into satellites/isolated and luminous/dark:

In [ ]:
def arrange_radius_array(r):
    r = r * units.cm.to(units.kpc)
    r = np.concatenate((np.sort(r), np.array([10000]))) 
    return r

def construct_count_array(size):
    return np.concatenate((np.arange(1, size+1), 
                           np.array([size])))

In [ ]:
for key, sim_data in data.items():    
    # Get data:
    snap = sim_data["snapshot"]
    cops = snap.get_subhalos("CentreOfPotential")
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_compute.mask_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_GN":
        masks_sat, mask_isol = dataset_compute.mask_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    
    mask_m31_lum = np.logical_and.reduce([masks_sat[0], mask_lum, mask_nonzero_vmax])
    mask_m31_dark = np.logical_and.reduce([masks_sat[0], mask_dark, mask_nonzero_vmax])
    mask_mw_lum = np.logical_and.reduce([masks_sat[1], mask_lum, mask_nonzero_vmax])
    mask_mw_dark = np.logical_and.reduce([masks_sat[1], mask_dark, mask_nonzero_vmax])
    mask_isol_lum = np.logical_and.reduce([mask_isol, mask_lum, mask_nonzero_vmax])
    mask_isol_dark = np.logical_and.reduce([mask_isol, mask_dark, mask_nonzero_vmax])
    
    # In case M31 and MW are identified as the same halo, remove intersection:
    mask_mw_lum = np.logical_and(mask_mw_lum, np.logical_not(mask_m31_lum))
    mask_mw_dark = np.logical_and(mask_mw_dark, np.logical_not(mask_m31_dark))
   
    # Compute distances to M31 and MW:
    m31_cop = cops[snap.index_of_halo(sim_data["M31_identifier"][0],
                                      sim_data["M31_identifier"][1])]
    dist_to_m31 = dataset_compute.distance_to_point(snap, m31_cop)
    mw_cop = cops[snap.index_of_halo(sim_data["MW_identifier"][0],
                                     sim_data["MW_identifier"][1])]
    dist_to_mw = dataset_compute.distance_to_point(snap, mw_cop)

    # ... and add satellites to the data dictionary:
    sat_dist_lum = arrange_radius_array(
        np.concatenate([dist_to_m31[mask_m31_lum], dist_to_mw[mask_mw_lum]]))
    sat_dist_dark = arrange_radius_array(
        np.concatenate([dist_to_m31[mask_m31_dark], dist_to_mw[mask_mw_dark]]))
    idx = np.arange(sat_dist_lum.size)
    
    sim_data["satellites"] = \
    {"radius": {"luminous": sat_dist_lum,
                "dark": sat_dist_dark
               },
     "sat_count": {"luminous": construct_count_array(
         np.sum(np.logical_or(mask_m31_lum, mask_mw_lum))),
                   "dark": construct_count_array(
         np.sum(np.logical_or(mask_m31_dark, mask_mw_dark)))
                  }
    }
    
    # Compute distances to LG centre:
    lg_centre = dataset_compute.compute_LG_centre(snap,
                                                  sim_data["M31_identifier"],
                                                  sim_data["MW_identifier"])
    dist_to_lg = dataset_compute.distance_to_point(snap, lg_centre)

    # ... and add isolated galaxies to the data dictionary:
    isol_dist_lum = arrange_radius_array(dist_to_lg[mask_isol_lum])
    isol_dist_dark = arrange_radius_array(dist_to_lg[mask_isol_dark])
    
    sim_data["isolated"] = \
    {"radius": {"luminous": isol_dist_lum,
                "dark": isol_dist_dark
               },
     "sat_count": {"luminous": construct_count_array(np.sum(mask_isol_lum)),
                   "dark": construct_count_array(np.sum(mask_isol_dark))
                  }
    }


## Plot

In [ ]:
# Set some parameters:
x_down_sat = 0; x_up_sat = 400
y_down_sat = 1; y_up_sat = 500

x_down_isol = 0; x_up_isol = 2100
y_down_isol = 1; y_up_isol = 500

# Set colors:
color = ["black", "red", "blue", "green"]

In [ ]:
# Construct saving location:
filename = 'sat_and_isol_r_dist_{}'.format(distinction)
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis for satellites:
axes[0].set_xlim(x_down_sat, x_up_sat)
axes[0].set_ylim(y_down_sat, y_up_sat)
axes[0].set_xlabel('$r_\mathrm{central}[\mathrm{kpc}]$', fontsize=16)
axes[0].set_ylabel('$N(<r_\mathrm{central})$', fontsize=16)
axes[0].set_yscale('log')
axes[0].yaxis.set_ticks_position("both")

# Set axis for isolated galaxies:
axes[1].set_xlim(x_down_isol, x_up_isol)
axes[1].set_ylim(y_down_isol, y_up_isol)
axes[1].set_xlabel('$r_\mathrm{LG}}[\mathrm{kpc}]$', fontsize=16)
axes[1].set_ylabel('$N(<r_\mathrm{LG})$', fontsize=16)
axes[1].set_yscale('log')
axes[1].yaxis.set_ticks_position("both")
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['satellites']['radius']['luminous']
    y = entry['satellites']['sat_count']['luminous']
    axes[0].plot(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['satellites']['radius']['dark']
    y = entry['satellites']['sat_count']['dark']
    axes[0].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))
    
    x = entry['isolated']['radius']['luminous']
    y = entry['isolated']['sat_count']['luminous']
    axes[1].plot(x, y, c=color[i], label='{} luminous'.format(name))
    x = entry['isolated']['radius']['dark']
    y = entry['isolated']['sat_count']['dark']
    axes[1].plot(x, y, c=color[i], linestyle='--', label='{} dark'.format(name))

axes[0].legend(loc='upper right')
plt.tight_layout()

plt.savefig(filename, dpi=200)